In [1]:
!pip install mysql.connector

Defaulting to user installation because normal site-packages is not writeable
  Using cached mysql-connector-2.2.9.tar.gz (11.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  ERROR: Error [WinError 225] No se pudo completar la operación porque el archivo contiene un virus o software potencialmente no deseado while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] No se pudo completar la operación porque el archivo contiene un virus o software potencialmente no deseado



In [2]:
pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1414]:
#1. IMPORTAMOS LIBRERIAS NECESARIAS

import mysql.connector
import pandas as pd


In [1415]:
# CARGAMOS LA DATA

ruta_archivo_excel="Datos sin nulos.xlsx"
df=pd.read_excel(ruta_archivo_excel)
df.head()

,Unnamed: 0,Nombre,Edad,Puntuacion,Sexo
0,0,Anthony Wallace,46,86,M
1,1,Steven Barker,25,67,F
2,2,Barry Graham,56,82,M
3,3,Austin Jackson,40,92,M
4,6,Zachary Sampson,20,85,M


In [1416]:
# CONEXION A LA BASE DE DATOS

# METODO PARA CONECTAR A LA BASE DE DATOS

def establecer_conexion(host, user, password, database):
  try:
      conexion = mysql.connector.connect(
          host = host,
          user = user,
          password = password,
          database = database)
      return conexion
  except mysql.connector.Error as err:
        print(f"Error:{ err}")
        return None

In [1417]:
conexion = establecer_conexion("localhost", "root", "", "estudiante")
#Comprobación de la conexión
conexion.is_connected()

True

In [1418]:
# ELIMINAR LOS DATOS DE LA BASE DE DATOS

def eliminar_datos_base(conexion):
    try:
        # Desactivar las restricciones de clave externa para permitir la eliminación
        #with conexion.cursor() as cursor:
            #cursor.execute("SET FOREIGN_KEY_CHECKS=0;")
        cursor=conexion.cursor()
        # Eliminar datos de las tablas
        delete_queries = [
            'DELETE FROM estudiante;',
            'DELETE FROM genero;',
            'DELETE FROM estudiante_stg;',]

        for query in delete_queries:
            cursor.execute(query)

        # Reiniciar los contadores de autoincremento
        reset_auto_increment_queries = [
            'ALTER TABLE estudiante AUTO_INCREMENT = 1;',
            'ALTER TABLE genero AUTO_INCREMENT = 1;',
            'ALTER TABLE estudiante_stg AUTO_INCREMENT = 1;']

        for query in reset_auto_increment_queries:
            cursor.execute(query)

        # Confirmar los cambios
        conexion.commit()

        print("Base de datos limpiada exitosamente.")

    except Exception as e:
        print(f"Error al limpiar la base de datos: {e}")

    finally:
        # Reactivar las restricciones de clave externa
        print(".....")

In [1390]:
eliminar_datos_base(conexion)

Base de datos limpiada exitosamente.
.....


In [1419]:
# INSERTAR DATOS EN LA TABLA ESTUDIANTE_STG

def insert_estudiante_stg(conexion, df):
    cursor = conexion.cursor()

    insert_query = "INSERT INTO estudiante_stg(Nombre,Edad,Puntuacion,Genero) VALUES (%s, %s, %s, %s)"

    try:
        for index, row in df.iterrows():
            estudiante_data = (row['Nombre'], row['Edad'], row['Puntuacion'], row['Sexo'])
            cursor.execute(insert_query, estudiante_data)

        conexion.commit()
        print(f"Datos insertados correctamente")

    except Exception as e:
        print(f"Error al insertar datos: {e}")

    finally:
        cursor.close()

In [1420]:
insert_estudiante_stg(conexion, df)

Datos insertados correctamente


In [1421]:
# LIMPIAR DATOS INNECESARIOS DE ESTUDIANTE

def limpiar_datos_estudiante_stg(conexion):
    try:
        # Desactivar las restricciones de clave externa para permitir la eliminación
        with conexion.cursor() as cursor:
            cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

        # Eliminar datos de las tablas
        delete_queries = [
            'delete from estudiante_stg where nombre=0;',
            'delete from estudiante_stg where edad like"%Marca%no%registrada%";',
            'delete from estudiante_stg where puntuacion like"%Categoria%no%registrada%";',
            'delete from estudiante_stg where genero="Noid"'        
        ]

        for query in delete_queries:
            with conexion.cursor() as cursor:
                cursor.execute(query)

        # Confirmar los cambios
        conexion.commit()

        print("Datos tabla estudiante stg limpiada exitosamente.")

    except Exception as e:
        print(f"Error al limpiar la base de datos: {e}")

    finally:
        # Reactivar las restricciones de clave externa
        with conexion.cursor() as cursor:
            cursor.execute("SET FOREIGN_KEY_CHECKS=1;")

In [1422]:
limpiar_datos_estudiante_stg(conexion)

Datos tabla estudiante stg limpiada exitosamente.


In [1423]:
# OBTENER VALORES DISTINTOS Y ASIGNAR A LAS RESPECTIVAS TABLAS 

def obtener_distinct_values(conexion, column_name):
    try:
        with conexion.cursor() as cursor:
            select_query = f"SELECT DISTINCT({column_name}) FROM estudiante_stg"
            cursor.execute(select_query)
            distinct_values = cursor.fetchall()
            print(f"Datos extraidos correctamente de: {column_name}")
            return tuple(value[0] for value in distinct_values)

    except Exception as e:
        print(f"Error al obtener valores distintos: {e}")
        return []

In [1424]:
distinct_values_estudiante=obtener_distinct_values(conexion, "nombre")

Datos extraidos correctamente de: nombre


In [1425]:
distinct_values_edad=obtener_distinct_values(conexion, "edad")

Datos extraidos correctamente de: edad


In [1426]:
distinct_values_puntuacion=obtener_distinct_values(conexion, "puntuacion")

Datos extraidos correctamente de: puntuacion


In [1427]:
distinct_values_genero=obtener_distinct_values(conexion, "genero")

Datos extraidos correctamente de: genero


In [1428]:
# INSERTAR DATOS EXTRAIDOS EN LAS RESPECTIVAS TABLAS DE LA BASE DE DATOS

def insert_distinct_values_into_table(conexion, distinct_values, table_name, column_name):
        try:
            with conexion.cursor() as cursor:
                insert_query = f"INSERT INTO {table_name} ({column_name}) VALUES (%s)"
                for value in distinct_values:
                    cursor.execute(insert_query, (value,))

                conexion.commit()
                print(f"Datos insertados correctamente en {table_name}.")

        except Exception as e:
            print(f"Error al insertar datos en {table_name}: {e}")  

In [1429]:
insert_distinct_values_into_table(conexion, distinct_values_generos, "genero","descripcion_genero")

Datos insertados correctamente en genero.


In [1460]:
def obtener_valores_estudiante(conexion):
    try:
        with conexion.cursor() as cursor:
            select_query = '''
            SELECT estudiante.genero, genero.id_genero
            FROM genero
            INNER JOIN estudiante ON estudiante.genero = genero.descripcion_genero
            '''
            cursor.execute(select_query)
            values = cursor.fetchall()
            return tuple(values)

    except Exception as e:
        print(f"Error al obtener valores distintos: {e}")
        return []


In [1461]:
valores=obtener_valores_estudiante(conexion)

In [1458]:
# INSERTAR DATOS EN LA TABLA ESTUDIANTE

def insert_estudiante(conexion, df):
    cursor = conexion.cursor()

    insert_query = "INSERT INTO estudiante(Nombre,Edad,Puntuacion,Genero) VALUES (%s, %s, %s, %s)"

    try:
        for index, row in df.iterrows():
            estudiante_data = (row['Nombre'], row['Edad'], row['Puntuacion'], row['Sexo'])
            cursor.execute(insert_query, estudiante_data)

        conexion.commit()
        print(f"Datos insertados correctamente")

    except Exception as e:
        print(f"Error al insertar datos: {e}")

    finally:
        cursor.close()

In [1459]:
insert_estudiante(conexion, df)

Datos insertados correctamente
